In [1]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from itertools import combinations 
from numpy import savetxt
import random

2022-10-20 00:27:45.028120: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-20 00:27:45.028197: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

In [10]:
cols = [
    'carga_subrede_138kv',
    'pot_eolica_subrede_138kv',
    'pot_solar_subrede_138kv',
    'carga_subrede_230kv',
    'pot_eolica_subrede_230kv',
    'pot_solar_subrede_230kv',
    'phase_shift_transformador3_24',
    'relacao_tranformacao_transformador3_24'
]
input_df = pd.read_csv("data/input_data_nn_T.csv", sep=";")[cols]
output_df = pd.read_csv("data/output_data_nn_T.csv", sep=";")
df = pd.concat([input_df, output_df], axis=1)
# df = df.loc[~(df==0).all(axis=1)].reset_index()
# print(output_df.describe().loc[['mean', 'std', 'min', 'max']].transpose().to_latex())

In [36]:
df.tail()

,carga_subrede_138kv,pot_eolica_subrede_138kv,pot_solar_subrede_138kv,carga_subrede_230kv,pot_eolica_subrede_230kv,pot_solar_subrede_230kv,phase_shift_transformador3_24,relacao_tranformacao_transformador3_24,status_1_gerador_subrede138kv,status_2_gerador_subrede138kv,...,status_21_linha_subrede_230kv,tensao_barramento11,tensao_barramento12,tensao_barramento24,pot_ativa_inj_barramento11,pot_ativa_inj_barramento12,pot_ativa_inj_barramento24,pot_reativa_inj_barramento11,pot_reativa_inj_barramento12,pot_reativa_inj_barramento24
183955,650.684982,0.0,384.262473,782.267711,0.990714,39.427453,0,1.00,1,1,...,1,0.998002,1.000294,0.992269,6.672915,58.770996,26.227602,36.520962,41.345638,11.270963
183956,650.684982,0.0,384.287671,782.267711,0.991085,39.440497,0,1.01,1,1,...,1,0.998026,1.000381,0.989495,6.520626,58.674342,26.467219,35.826736,40.798340,15.209594
183957,650.684982,0.0,384.324570,782.267711,0.991548,39.455575,0,1.02,1,1,...,1,0.998073,1.000487,0.986778,6.369393,58.578784,26.700097,35.144730,40.252193,19.115678
183958,650.684982,0.0,384.417204,782.267711,0.992645,39.490751,0,1.04,1,1,...,1,0.998223,1.000739,0.981496,6.076092,58.394902,27.149829,33.826118,39.174355,26.834484
183959,650.684982,0.0,384.453070,782.267711,0.993030,39.510302,0,1.05,1,1,...,1,0.998335,1.000896,0.978938,5.941975,58.312190,27.371701,33.203552,38.656658,30.648606


In [12]:
def augment(x, y, augmenter = None):
    x, augmenter = _add_interactions(x, augmenter)
    return x, y, augmenter, x.columns

def _add_interactions(df, augmenter):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]

    if augmenter:
        df = augmenter.transform(df)
    else:
        augmenter = PolynomialFeatures(interaction_only=True, include_bias=False)
        df = augmenter.fit_transform(df)

    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df, augmenter

## With augmentation

In [4]:
def run_main(train_X, test_X, train_Y, test_Y, cur_fold, cur_run):
    data = [train_X, test_X, train_Y, test_Y]
    predicted_df = pd.DataFrame(columns=test_Y.columns, index=test_Y.index.values)
    new_df = test_Y.copy()

    cols_pred = ['tensao_barramento11']
    model_id = 'tensao_11'
    predicted_11 = run(new_df, cols_pred, data, cur_fold, cur_run)

    cols_pred = ['tensao_barramento12']
    model_id = 'tensao_12'
    predicted_12 = run(new_df, cols_pred, data, cur_fold, cur_run)

    cols_pred = ['tensao_barramento24']
    model_id = 'tensao_24'
    predicted_24 = run(new_df, cols_pred, data, cur_fold, cur_run)
    
    new_df['tensao_barramento11'] = predicted_11
    new_df['tensao_barramento12'] = predicted_12
    new_df['tensao_barramento24'] = predicted_24
    predicted_df['tensao_barramento11'] = predicted_11
    predicted_df['tensao_barramento12'] = predicted_12
    predicted_df['tensao_barramento24'] = predicted_24
        
    cols_pred = ['pot_ativa_inj_barramento11']
    model_id = 'pot_ativa_inj_barramento11'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_ativa_inj_barramento12']
    model_id = 'pot_ativa_inj_barramento12'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_ativa_inj_barramento24']
    model_id = 'pot_ativa_inj_barramento24'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_reativa_inj_barramento11']
    model_id = 'pot_reativa_inj_barramento11'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento12']
    model_id = 'pot_reativa_inj_barramento12'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento24']
    model_id = 'pot_reativa_inj_barramento24'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions    

    predicted_df.to_csv(f'results-T/{cur_run}-{cur_fold}.csv')

In [5]:
def run(df, cols_pred, data, idx, cur_run):
    trainX, testX, trainY, testY = data

    trainY = trainY[cols_pred]
    testY = testY[cols_pred]

    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.transform(testX)
    
    print(f" ========== MODEL {cols_pred[0]} {idx} {cur_run} ============")
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)

    model = Sequential()
    model.add(Input(shape=(trainX.shape[1],)))
    model.add(Dense(trainX.shape[-1] ** 2, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(trainX.shape[-1], activation = 'relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(len(cols_pred)))

    model.compile(loss = 'mae', optimizer = 'adam')
    model.fit(
        trainX, trainY,
        batch_size = 50, epochs = 20,
        callbacks = [es],
        validation_data = (testX, testY),
        verbose=1
    )

    _predictions = model.predict(testX)
    rmse = sqrt(mean_squared_error(testY, _predictions))
    print(f"RMSE: {rmse}")
    
    return _predictions  

In [13]:
for i in range(1, 2):
    cur_fold = 1
    kfold = KFold(n_splits=10, shuffle=True)

    X, Y, _, _, = augment(input_df, output_df)
    for train_ix, test_ix in kfold.split(X, Y):
        train_X, test_X = X.loc[train_ix], X.loc[test_ix]
        train_y, test_y = Y.loc[train_ix], Y.loc[test_ix]

        run_main(train_X, test_X, train_y, test_y, cur_fold, i)

        cur_fold += 1

 ========== MODEL tensao_barramento11 1 1 ============


2022-10-20 00:38:35.899530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-20 00:38:35.899560: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-20 00:38:35.899588: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (schlickmann): /proc/driver/nvidia/version does not exist
2022-10-20 00:38:35.900498: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3306/3312 [============================>.] - ETA: 0s - loss: 0.0556WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f6783c80d30> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 

3312/3312 [==============================] - 7s 2ms/step - loss: 0.0016 - val_loss: 0.0012
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
575/575 [==============================] - 1s 810us/step
RMSE: 0.002150139014241044
 ========== MODEL tensao_barramento24 1 1 ============
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full o

3312/3312 [==============================] - 7s 2ms/step - loss: 11.3965 - val_loss: 3.2732
Epoch 4/20
3312/3312 [==============================] - 7s 2ms/step - loss: 11.1956 - val_loss: 3.4172
Epoch 5/20
3312/3312 [==============================] - 7s 2ms/step - loss: 11.0729 - val_loss: 2.7259
Epoch 6/20
3312/3312 [==============================] - 7s 2ms/step - loss: 10.8944 - val_loss: 3.0645
Epoch 7/20
3312/3312 [==============================] - 9s 3ms/step - loss: 10.7770 - val_loss: 5.7861
Epoch 8/20
3312/3312 [==============================] - 9s 3ms/step - loss: 10.6151 - val_loss: 3.4599
Epoch 9/20
3312/3312 [==============================] - 9s 3ms/step - loss: 10.4257 - val_loss: 1.8920
Epoch 10/20
3312/3312 [==============================] - 9s 3ms/step - loss: 10.3593 - val_loss: 1.2005
Epoch 11/20
3312/3312 [==============================] - 9s 3ms/step - loss: 10.2214 - val_loss: 2.9209
Epoch 12/20
3312/3312 [==============================] - 9s 3ms/step - loss: 10.09

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3312/3312 [==============================] - 7s 2ms/step - loss: 0.0542 - val_loss: 0.0029
Epoch 2/20
3312/3312 [==============================] - 7s 2ms/step - loss: 0.0029 - val_loss: 0.0025
Epoch 3/20
3312/3312 [==============================] - 7s 2ms/step - loss: 0.0026 - val_loss: 0.0021
Epoch 4/20
3312/3312 [==============================] - 7s 2ms/step - loss: 0.0025 - val_loss: 0.0020
Epoch 5/20
3312/3312 [==============================] - 7s 2ms/step - loss: 0.0024 - val_loss: 0.0022
Epoch 6/20
3312/3312 [==============================] - 7s 2ms/step - loss: 0.0024 - val_loss: 0.0015
Epoch 7/20
3312/3312 [==============================] - 7s 2ms/step - loss: 0.0023 - val_loss: 0

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3297/3312 [============================>.] - ETA: 0s - loss: 13.4195WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f677fec6b80> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'ga

3312/3312 [==============================] - 7s 2ms/step - loss: 10.0043 - val_loss: 2.0477
Epoch 15/20
3312/3312 [==============================] - 7s 2ms/step - loss: 9.8390 - val_loss: 4.0827
Epoch 16/20
3312/3312 [==============================] - 7s 2ms/step - loss: 9.7288 - val_loss: 1.9977
Epoch 17/20
3312/3312 [==============================] - 7s 2ms/step - loss: 9.5902 - val_loss: 1.6138
Epoch 18/20
3312/3312 [==============================] - 7s 2ms/step - loss: 9.4066 - val_loss: 2.1769
Epoch 19/20
3312/3312 [==============================] - 7s 2ms/step - loss: 9.2911 - val_loss: 3.2451
Epoch 20/20
3312/3312 [==============================] - 7s 2ms/step - loss: 9.1565 - val_loss: 1.2787
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do

3312/3312 [==============================] - 7s 2ms/step - loss: 3.7914 - val_loss: 2.1846
Epoch 2/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.7437 - val_loss: 1.6530
Epoch 3/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.5796 - val_loss: 1.5703
Epoch 4/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.4734 - val_loss: 1.5998
Epoch 5/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.3621 - val_loss: 1.5362
Epoch 6/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.3019 - val_loss: 1.2401
Epoch 7/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.2599 - val_loss: 1.3358
Epoch 8/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.2171 - val_loss: 1.4253
Epoch 9/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.1898 - val_loss: 1.2397
Epoch 10/20
3312/3312 [==============================] - 7s 2ms/step - loss: 2.1740 - val_los

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3307/3312 [============================>.] - ETA: 0s - loss: 2.7870WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f67707c3d30> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gas

In [12]:
indexes = np.array(df.index.tolist())
random.shuffle(indexes)
ids_to_pred = np.split(indexes, 4)

results_all = []
for i in range(5):
    results_per_ids = []
    for idx2 in range(len(ids_to_pred)):
        results = []
        run_main(df, i, idx2, ids_to_pred[idx2])
        results_per_ids.append(results)
        
    results_all.append(results_per_ids)

 ========== MODEL tensao_11 0 ============
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2755/2760 [============================>.] - ETA: 0s - loss: 0.0594WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f0fa51a2790> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSI

2760/2760 [==============================] - 7s 3ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 15/20
2760/2760 [==============================] - 7s 3ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 16/20
2760/2760 [==============================] - 7s 3ms/step - loss: 0.0022 - val_loss: 0.0014
Epoch 17/20
2760/2760 [==============================] - 7s 3ms/step - loss: 0.0022 - val_loss: 0.0013
Epoch 18/20
2760/2760 [==============================] - 7s 3ms/step - loss: 0.0022 - val_loss: 0.0013
Epoch 19/20
2760/2760 [==============================] - 7s 3ms/step - loss: 0.0022 - val_loss: 0.0012
Epoch 20/20
2760/2760 [==============================] - 7s 3ms/step - loss: 0.0022 - val_loss: 0.0013
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_

2760/2760 [==============================] - 19s 7ms/step - loss: 10.9967 - val_loss: 6.1084
Epoch 2/20
2760/2760 [==============================] - 18s 7ms/step - loss: 9.2208 - val_loss: 4.9627
Epoch 3/20
2760/2760 [==============================] - 18s 7ms/step - loss: 9.0009 - val_loss: 3.3999
Epoch 4/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.8510 - val_loss: 2.9795
Epoch 5/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.7376 - val_loss: 4.3349
Epoch 6/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.6492 - val_loss: 3.0154
Epoch 7/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.5454 - val_loss: 2.5340
Epoch 8/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.5057 - val_loss: 1.9632
Epoch 9/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.4286 - val_loss: 2.0032
Epoch 10/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.391

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2760/2760 [==============================] - 19s 7ms/step - loss: 9.9707 - val_loss: 5.2689
Epoch 2/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.3201 - val_loss: 3.6032
Epoch 3/20
2760/2760 [==============================] - 18s 7ms/step - loss: 8.1310 - val_loss: 4.7174
Epoch 4/20
2760/2760 [==============================] - 18s 7ms/step - loss: 7.9814 - val_loss: 4.3030
Epoch 5/20
2760/2760 [==============================] - 18s 7ms/step - loss: 7.8029 - val_loss: 2.7581
Epoch 6/20
2760/2760 [==============================] - 18s 7ms/step - loss: 7.7374 - val_loss: 2.3675
Epoch 7/20
2760/2760 [==============================] - 18s 7ms/step - loss: 7.5987 - val_

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2756/2760 [============================>.] - ETA: 0s - loss: 1.9901WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f102a18c790> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gas

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2757/2760 [============================>.] - ETA: 0s - loss: 11.7474WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f0f6038bc10> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'ga

2760/2760 [==============================] - 18s 7ms/step - loss: 1.2415 - val_loss: 3.0384
Epoch 11/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.2239 - val_loss: 2.7967
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1438/1438 [==============================] - 2s 2ms/step
RMSE: 6.361133114775445
 ========== MODEL pot_reativa_inj_barramento24 4 ============
Epoch 1/20
Please report this to the TensorFlow team. W

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2752/2760 [============================>.] - ETA: 0s - loss: 0.0575WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f1012185af0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gas

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1438/1438 [==============================] - 2s 2ms/step
RMSE: 4.0262614150102545
 ========== MODEL pot_ativa_inj_barramento12 4 ============
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorat

2760/2760 [==============================] - 19s 7ms/step - loss: 2.0464 - val_loss: 2.3548
Epoch 2/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.5374 - val_loss: 2.3070
Epoch 3/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.4455 - val_loss: 2.3758
Epoch 4/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.3914 - val_loss: 2.4782
Epoch 5/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.3686 - val_loss: 2.7893
Epoch 6/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.3381 - val_loss: 2.7612
Epoch 7/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.3122 - val_loss: 2.8754
Epoch 8/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.3046 - val_loss: 2.9018
Epoch 9/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.2792 - val_loss: 3.0641
Epoch 10/20
2760/2760 [==============================] - 18s 7ms/step - loss: 1.2683

In [21]:
for _result in results_all:
    print(results_all.index(_result))
    for fold in _result:
        print(f"{_result.index(fold)}: {sum(fold)}")

0
0: 32.33497086128333
1: 36.889005154036376
2: 36.68749314981343
3: 31.932388827029182
1
0: 32.26412368037137
1: 31.990872924834257
2: 36.0298650015969
3: 32.30431549681663
2
0: 31.72844917901247
1: 29.84034210963254
2: 34.37873809844469
3: 38.184718100379435
3
0: 34.15181711482428
1: 30.558933149246954
2: 30.1250874789621
3: 31.1846092629998
4
0: 34.86386086977219
1: 28.93438134306144
2: 36.44611309999486
3: 33.4754096497843
5
0: 37.116041669193386
1: 37.296140359486756
2: 30.919280629845613
3: 39.44656374003831
6
0: 31.38229179727558
1: 30.653259296110438
2: 31.444674673125057
3: 36.76260480496647
7
0: 28.349547201857266
1: 33.73417886152881
2: 39.737148167795624
3: 37.59890026817541
8
0: 29.559890665146096
1: 40.83094120187875
2: 26.866222288396877
3: 37.813916555230335
9
0: 38.21746676037965
1: 37.003628372655385
2: 47.06004230959253
3: 34.13258223262406


In [26]:
print(f"Fold 0: {results_all[7][0]}")
print(f"Fold 1: {results_all[4][1]}")
print(f"Fold 2: {results_all[8][2]}")
print(f"Fold 3: {results_all[3][3]}")

Fold 0: [0.005618806104382909, 0.006374496128408189, 0.005003273984089787, 3.4735610445018055, 5.057355698378199, 3.6107973146928116, 7.466659649968084, 6.293739685672916, 2.4304372324265673]
Fold 1: [0.004912833436191945, 0.006315874222206748, 0.0050400543592956865, 5.882427740243566, 3.5072674248462787, 3.786373119903367, 6.751693446504084, 6.168161926473203, 2.822188923073245]
Fold 2: [0.004500589408310584, 0.004620877592182861, 0.005015842571451259, 3.953451113791048, 4.887246335988537, 4.617532544758241, 5.595597117037946, 5.241330146546544, 2.5569277207026158]
Fold 3: [0.005904257261356303, 0.004717165985727671, 0.005687405161361025, 3.2969980512393264, 5.985023419883928, 3.5217441168282497, 8.310837484354026, 6.692027891640731, 3.3616694706450936]


In [40]:
print(f"RMSE tensao_barramento11: {sqrt(mean_squared_error(df['tensao_barramento11'], df_best_predicted['tensao_barramento11']))}")
print(f"RMSE tensao_barramento12: {sqrt(mean_squared_error(df['tensao_barramento12'], df_best_predicted['tensao_barramento12']))}")
print(f"RMSE tensao_barramento24: {sqrt(mean_squared_error(df['tensao_barramento24'], df_best_predicted['tensao_barramento24']))}")

print(f"RMSE pot_ativa_inj_barramento11: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento11'], df_best_predicted['pot_ativa_inj_barramento11']))}")
print(f"RMSE pot_ativa_inj_barramento12: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento12'], df_best_predicted['pot_ativa_inj_barramento12']))}")
print(f"RMSE pot_ativa_inj_barramento24: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento24'], df_best_predicted['pot_ativa_inj_barramento24']))}")


RMSE tensao_barramento11: 0.005263612667191903
RMSE tensao_barramento12: 0.005570651504552013
RMSE tensao_barramento24: 0.005194712520332189
RMSE pot_ativa_inj_barramento11: 0.0
RMSE pot_ativa_inj_barramento12: 0.0
RMSE pot_ativa_inj_barramento24: 0.0
